# Gplate for slab dataset

In this notebook, I use gplately to extract slab dataset, resample and plot the results

## Prerequisite

- Install the gplately package in a conda environment. Refer to their home page and there installation link [https://github.com/GPlates/gplately](https://github.com/GPlates/gplately)
- Download this package and set path to the installation directory

In [ ]:
# use the environment of py-gplate
import sys
import gplately
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
from plate_model_manager import PlateModelManager
from shutil import rmtree

# Include this pakage
# change to your download path
HaMaGeoLib_DIR = "/home/lochy/ASPECT_PROJECT/HaMaGeoLib"
if os.path.abspath(HaMaGeoLib_DIR) not in sys.path:
    sys.path.append(os.path.abspath(HaMaGeoLib_DIR))

from hamageolib.research.haoyuan_3d_subduction.gplately_utilities import GPLOTTER, GPLATE_PROCESS
from hamageolib.utils.exception_handler import my_assert

# Set up

In [ ]:
# enter the directory of the plate reconstruction files
case_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction/gplate_dataset_09202025"
if not os.path.isdir(case_dir):
  os.mkdir(case_dir)
if not os.path.isdir(os.path.join(case_dir, "img")):
  os.mkdir(os.path.join(case_dir, "img"))
csv_dir = os.path.join(case_dir, "csv")
if not os.path.isdir(csv_dir):
  os.mkdir(csv_dir)

# assign a reconstruction time
model_name = "Muller2019"
reconstruction_time=0 # time of reconstruction, must be integar
assert(type(reconstruction_time) == int)
anchor_plate_id = 0 # anchor plate id: 0 - Africa

# set up a directory to output for every step
img_dir = os.path.join(os.path.join(case_dir, "img", "%05dMa" % reconstruction_time))
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

GplateP = GPLATE_PROCESS(case_dir)

# Import a lookup file for slab names

In [ ]:
# parse the name lookup file
# These files are exported from gplate (GUI)
parse_name_lookup = False
if parse_name_lookup:
    from hamageolib.research.haoyuan_3d_subduction.gplately_utilities import read_subduction_reconstruction_data

    subduction_name_lookup_file = os.path.join(case_dir, "Muller_etal_2019_PlateBoundaries_no_topologies",\
                                              "reconstructed_%.2fMa.xy" % float(reconstruction_time))
    name_lookups = read_subduction_reconstruction_data(subduction_name_lookup_file)

## Lookup for a key word

In [ ]:
lookup_key_word = False

if parse_name_lookup and lookup_key_word:

    # print(name_lookups["trench_names"]) # debug

    keyword = "ryu"

    matching_indices = [i for i, name in enumerate(name_lookups["trench_names"]) if keyword.lower() in name.lower()]
    for index in matching_indices:
        print(index)
        print("name: ", name_lookups["trench_names"][index])
        print("id: ", name_lookups["trench_pids"][index])
        print("")

## lookup by trench pids

In [ ]:
lookup_by_trench_pid = False

if parse_name_lookup and lookup_by_trench_pid:

    pid_lookup_list = [12001, 686, 736, 651, 669, 612, 678, 648, 659, 699, 111, 406, 413, 2000, 201, 2031, 2011, 815, 821]

    for index, pid in enumerate(name_lookups["trench_pids"]):
        if pid in pid_lookup_list:
            print("pid = %d, name = %s" % (pid, name_lookups["trench_names"][index]))

# Run in series

In [ ]:
run_in_series = True

## Import reconstruction dataset

In [ ]:
if run_in_series:
    GplateP.reconstruct(model_name, reconstruction_time, anchor_plate_id)

## Add age

In [ ]:
if run_in_series:
    GplateP.add_age_raster()
    GplateP.export_csv("subduction_data", "ori.csv")

## Inspect and save results of every subduction

In [ ]:
inspect_all_slabs = False
inspect_all_slabs_in_separate_plots = False

if run_in_series and inspect_all_slabs:
    GplateP.save_results_ori(inspect_all_slabs_in_separate_plots)

## Make A Global / Regional Scheme

In [ ]:
# todo_plot

plot_original_results_paper = True

if plot_original_results_paper:

    GplateP.save_results_global_paper(projection_type="robinson")


## Resample the dataset

In [ ]:
resample_dataset = True

if run_in_series and resample_dataset:

    # parameters for resampling 
    arc_length_edge = 2.0; arc_length_resample_section = 2.0

    GplateP.resample_subduction(arc_length_edge, arc_length_resample_section)

## Inspect and save results of resampled dataset

In [ ]:
inspect_all_slabs_resampled = True
inspect_all_slabs_resampled_plot_individual = True

if run_in_series and resample_dataset and inspect_all_slabs_resampled:

    GplateP.save_results_resampled(inspect_all_slabs_resampled_plot_individual)

## Analysis

In [ ]:
do_analysis = True

## Preparation

### define additional markers

In [ ]:
if run_in_series and do_analysis:

    from matplotlib.path import Path

    verts = [
        (0., 0.),   # Center
        (0.2, 0.6), # Upper arm
        (0., 0.),   # Center
        (0.4, 0.4), # Right diagonal
        (0., 0.),   # Center
        (0.6, 0.2), # Right arm
        (0., 0.),   # Center
        (0.4, -0.4),# Right down diagonal
        (0., 0.),   # Center
        (0.2, -0.6),# Bottom arm
        (0., 0.),   # Center
        (-0.4, -0.4),# Left down diagonal
        (0., 0.),   # Center
        (-0.6, -0.2),# Left arm
        (0., 0.),   # Center
        (-0.4, 0.4),# Left diagonal
        (0., 0.),   # Center
        (-0.2, 0.6),# Upper left arm
    ]
    codes = [Path.MOVETO] + [Path.LINETO, Path.MOVETO] * 8 + [Path.MOVETO]
    snowflake = Path(verts, codes)

    # Define vertices for two equilateral triangles
    vertices = [
        [0, 1], [-np.sqrt(3)/2, -0.5], [np.sqrt(3)/2, -0.5], [0, 1],  # First triangle
        [0, -1], [-np.sqrt(3)/2, 0.5], [np.sqrt(3)/2, 0.5], [0, -1]   # Second triangle
    ]
    # Flatten the vertices list for creating the Path
    vertices = np.array(vertices)
    # Define path codes (all 'LINETO' except the start 'MOVETO')
    codes = [Path.MOVETO] + [Path.LINETO] * (len(vertices) - 1)
    star_path = Path(vertices, codes)

### plot options for cases

Notes:

- The plot_by_name option plots the data points with their assigned names. These names have to be specified. Be default, set to False and the subducting_pid will be used to plot.

In [ ]:
if run_in_series and do_analysis:

    # Plot options
    plot_by_name = False  # True - by name; False - by pid
    
    # Retrieve the default color cycle
    default_colors = [color['color'] for color in plt.rcParams['axes.prop_cycle']]

    # assign plot options
    if plot_by_name:
        if reconstruction_time == 0:
            plot_options = \
            (
                (903, {"marker": 'o',  "markerfacecolor": "yellow", "name": "CAS"}),
                (511, {"marker": 's',  "markerfacecolor": "yellow", "name": "ANDA-SUM"}),
                (801, {"marker": 'd',  "markerfacecolor": "yellow", "name": "JAVA"}),
                (645, {"marker": snowflake,  "markerfacecolor": "black", "name": "SULA"}),
                (602, {"marker": 'x',  "markerfacecolor": "blue", "name": "LUZ"}),
                (608, {"marker": 's',  "markerfacecolor": 'c', "name": "PHIL"}),
                ({901: 699}, {"marker": '>',  "markerfacecolor": 'red', "name": "MAR"}),
                ({901: 659}, {"marker": 's',  "markerfacecolor": 'red', "name": "IZU"}),
                ({901: (601115.0, 601118.0)}, {"marker": '^',  "markerfacecolor": 'green', "name": "JAP"}),
                ({901: 406}, {"marker": 'v',  "markerfacecolor": 'green', "name": "KUKAM"}),
                ({901: 111}, {"marker": 'o',  "markerfacecolor": 'pink', "name": "ALE-ALA"}),
                ({901: (806, 821)}, {"marker": 'd',  "markerfacecolor": 'blue', "name": "TON-KERM"}),
                (909, {"marker": star_path,  "markerfacecolor": 'c', "name": "MEX"}),
                (911, {"marker": 'o',  "markerfacecolor": 'k', "name": "PER-NCHI-JUAN-SCHI"}),
                (802, {"marker": 'd',  "markerfacecolor": 'k', "name": "SSCHI-TBD"}),
                ({201: 2011}, {"marker": '+',  "markerfacecolor": 'pink', "name": "ANT"}),
                ({201: 815}, {"marker": '*',  "markerfacecolor": 'r', "name": "SAND"}),
                (1, {"marker": 'd',  "markerfacecolor": "r", "name": "RYU"})
            )
        else:
            raise NotImplementedError()
    else:
        plot_options = None

## Generate vs age plots

In [ ]:
analyze_age_combined = True

if run_in_series and do_analysis and analyze_age_combined:

    GplateP.plot_age_combined(resample_dataset, plot_options)

# Run_in_batch

In [ ]:
make_animation = True

if make_animation:
    start_time = 0.0
    end_time = 61.0
    interval = 1.0
    
    only_one_pid = None  # None - process all subductions; a number - only process with this subducting plid

    # resample parameters     
    arc_length_edge = 2.0; arc_length_resample_section = 2.0

    reconstruction_times = np.arange(start_time, end_time, interval)

## Run the python script

Run with the same option as above

    hamageolib/research/haoyuan_3d_subduction/scripts/Gplately_Slab_Dat_Batch.py

This loops the time of reconstructions, plot for the desinated regions

## Make animation

In [ ]:
if make_animation:
    import json
    from hamageolib.research.haoyuan_2d_subduction.workflow_scripts import create_avi_from_images
    
    ani_dir = os.path.join(case_dir, "animation")
    if not os.path.isdir(ani_dir):
        os.mkdir(ani_dir)

    # loop first to get pid and region dict
    pid_dict = {} # record existing pid in timestep
    region_dict = {} # record plot region for each subduction
    
    # todo_data
    # load pid dict from script output
    pid_filepath = os.path.join(case_dir, "pid_dict.json")
    assert(os.path.isfile(pid_filepath))

    with open(pid_filepath, 'r') as fin:
        pid_dict = json.load(fin)

    for subducting_pid, _times in pid_dict.items():

        # skip other pids if only one pid is requried
        if only_one_pid is not None:
            if int(subducting_pid) != only_one_pid:
                continue

        # create animation for each subduction zone
        ani_file_list = []

        for i, reconstruction_time in enumerate(_times):

            image_file = os.path.join("%05dMa" % int(reconstruction_time),\
                                   "resampled_edge%.1f_section%.1f" % (arc_length_edge, arc_length_resample_section),\
                                    "global_subduction_resampled_t%.2fMa_pid%06d.png" % (reconstruction_time, only_one_pid))

            my_assert(os.path.isfile(image_file), FileExistsError, "%s doesn't exist.")
        
            ani_file_list.append(image_file)

        ani_file_name = "pid%06d_%05dMa_%05dMa" % (int(subducting_pid), _times[0], _times[-1])
        output_file = os.path.join(ani_dir, "%s.avi" % ani_file_name)
        create_avi_from_images(ani_file_list, output_file, 1)